In [ ]:
!pip install freetype-py

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd

import scipy.io as sio
import librosa

import time

import skimage
import skimage.io
from skimage.transform import rescale

from scipy.io import wavfile
import os
import subprocess

import sys
sys.path.append("../../src")
from spectrogramtools import *
from stego import *
from swpowerstego import *
from imtools import *
from tsp import *
from curvature import *
import freetype

In [ ]:
## You might have to change this path based on your operating system
face = freetype.Face('/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf') 

X = get_string_freetype(face, "Barry", 200)
X = X/np.max(np.abs(X))
X -= np.mean(X, axis=0, keepdims=True)
plt.subplot(211)
plt.scatter(X[:, 0], X[:, 1], s=1)
plt.axis("equal")
X = get_tsp_tour(X)

plt.subplot(212)
plt.plot(X[:, 0], X[:, 1])
plt.axis("equal");

## Load in audio and compute best target

In [ ]:
# Audio parameters
x, sr = librosa.load("../../data/tunes/barry.mp3", sr=44100)
win_length = 1024
LT = 64
mag_idxs = [1, 2]
phase_idxs = mag_idxs
print([int(fi*sr/win_length) for fi in mag_idxs])

# Sliding window parameters
win = 64
fit_lam = 0.1
q = -1

sp = STFTPowerDisjoint(x, X, win_length, mag_idxs, phase_idxs, win, fit_lam, q, do_viterbi=True, LT=LT)
sp.plot(normalize=False)

sp.solve()
sp.plot(normalize=True)

In [ ]:
y = sp.reconstruct_signal()
ipd.Audio(y, rate=sr)

In [ ]:
prefix = "barry-text-2d"
filename = "{}.mp3".format(prefix)
wavfile.write("out.wav", sr, y)
if os.path.exists(filename):
    os.remove(filename)
subprocess.call("ffmpeg -i out.wav".split() + [filename], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
os.remove("out.wav")

z, sr = librosa.load(filename, sr=sr)
z_sp = STFTPowerDisjoint(z, X, win_length, mag_idxs, phase_idxs, win, fit_lam, q, do_viterbi=False, LT=LT)
z_sp.MagSolver.targets = sp.MagSolver.targets
snr = get_snr(x, z)
distortion = z_sp.get_distortion()[0]
ratio = snr/distortion

title = "snr = {:.3f}, distortion = {:.3f}".format(snr, distortion,)

plt.figure(figsize=(6, 6))
z_sp.plot(True, title)
plt.tight_layout()
plt.savefig("{}.png".format(prefix), facecolor='white')
